# Extract all Fundamental Data from Screener.in using Beautiful Soup and Selenium

In [20]:
import numpy as np # linear algebra
import pandas as pd # pandas for dataframe based data processing and CSV file I/O
import requests # for http requests
from bs4 import BeautifulSoup # for html parsing and scraping
import bs4
import json
from tidylib import tidy_document # for tidying incorrect html
import pandas as pd
import time
from selenium import webdriver
import pickle
import selenium.webdriver 

In [21]:
import pandas as pd
read_nifty=pd.read_csv('data/NSE_500_Output.csv')
#read_nifty.head()
#read_nifty = read_nifty[read_nifty.Industry != 'FINANCIAL SERVICES']
#read_nifty = read_nifty[read_nifty.Industry != 'CONSTRUCTION']
symbol_list=read_nifty['Symbol'].unique().tolist()
len(symbol_list)

483

In [22]:
# Update Chrome Driver Path for Selenium to extract data from Screener.in
chrome_driver_path='C:\\Users\\sashhar\\Desktop\\Quantopian\\driver\\chromedriver.exe'

# Code for Extracting Data from Screener.in for each Symbol

In [23]:
##################With Pickle Sabeeh1983
symbol_list=['INFY','RELIANCE']
z_file_standalone=pd.DataFrame()
q_file_standalone=pd.DataFrame()
y_file_standalone=pd.DataFrame()
s_file_standalone=pd.DataFrame()
q_file_name_standalone="data/Quarter_Ticker_nifty500_standalone"+'.csv'
y_file_name_standalone="data/Yearly_Ticker_nifty500_standalone"+'.csv'
z_file_name_standalone="data/Peer_Ticker_nifty500_standalone"+'.csv'
s_file_name_standalone='data/Share_Holding_nifty500_standalone'+'.csv'
from selenium import webdriver
folder_to_save='C:\\Users\\sashhar\\Desktop\\Quantopian\\Data\\'
count=1
for symbol in symbol_list:
    print("Now processing for Symbol: ",count,"  ", symbol)
    driver = webdriver.Chrome(chrome_driver_path)
    driver.get("http://www.screener.in") 
    cookies = pickle.load(open("cookie_use.pkl", "rb"))
    for cookie in cookies: 
        driver.add_cookie(cookie)
    url="https://www.screener.in/company/"+symbol
    driver.get(url)
    content = BeautifulSoup(driver.page_source, "html.parser")
    try:
        z2=content.find("section",{"id":'peers'}).find("div",{"id":'peers-table-placeholder'}).find_all('table')
        z3=pd.read_html(str(z2))[0]
        z3['Ticker']=symbol
        z3['Type']="Peer_Comparison"
        company_name=content.find("a",{"class":'active'}).text
        z3['Company_Name']=company_name
        #print(z3.head())
        z_file_standalone=z_file_standalone.append(z3)
    except:
        print("Error getting peers data for symbol:",symbol)
    try:
        quarter_income_table=content.find("section",{"id":'quarters'}).find_all('table')
        quarter_income_df = pd.read_html(str(quarter_income_table))[0].T
        quarter_income_df.columns = quarter_income_df.iloc[0]
        quarter_income_df=quarter_income_df.drop(quarter_income_df.index[0])
        quarter_income_df.index.names=['Date']
        quarter_income_df['Ticker']=symbol
        quarter_income_df['Type']="Quarterly_Income_Statement"
        q_file_standalone=q_file_standalone.append(quarter_income_df)
    except:
        print("Error Getting Quarter data for symbol:",symbol)
    try:
        shareholding_table=content.find("section",{"id":'shareholding'}).find_all('table')
        shareholding_table_df = pd.read_html(str(shareholding_table))[0].T
        shareholding_table_df.columns = shareholding_table_df.iloc[0]
        shareholding_table_df=shareholding_table_df.drop(shareholding_table_df.index[0])
        shareholding_table_df.index.names=['Date']
        shareholding_table_df['Ticker']=symbol
        shareholding_table_df['Type']="Shareholding_Pattern"  
        #shareholding_table_df['Company_Name']=company_name
        s_file_standalone=s_file_standalone.append(shareholding_table_df)
    except:
        print("Error Getting Shareholder data for symbol:", symbol)
    try:
        income_table=content.find("section",{"id":'profit-loss'}).find_all('table')
        income_statement_df = pd.read_html(str(income_table))[0]
        income_statement_df.columns.values[0] = "KPI"
        income_statement_df.set_index("KPI",inplace=True)
        column_list = income_statement_df.columns.tolist()
        if 'TTM' in column_list :
            TTM=income_statement_df['TTM']
            income_statement_df=income_statement_df.drop(columns=['TTM'])
            income_statement_df.rename(columns = lambda x: x.split(' ')[1], inplace=True)
            income_statement_df['TTM']=TTM
        else:
            income_statement_df.rename(columns = lambda x: x.split(' ')[1], inplace=True)
        income_statement_df['Ticker']=symbol
        income_statement_df['Type']="Income_Statement"
        income_statement_df=income_statement_df.loc[:,~income_statement_df.columns.duplicated()]
        #income_statement_df['Company_Name']=company_name
        y_file_standalone=y_file_standalone.append(income_statement_df)
    except:
        print("Error Getting Income Statement for symbol:",symbol)
    try:
        balance_sheet_table=content.find("section",{"id":'balance-sheet'}).find_all('table')
        balance_sheet_df = pd.read_html(str(balance_sheet_table))[0]
        balance_sheet_df.columns.values[0] = "KPI"
        balance_sheet_df.set_index("KPI",inplace=True)
        column_list = balance_sheet_df.columns.tolist()
        if 'TTM' in column_list :
            TTM=balance_sheet_df['TTM']
            balance_sheet_df=balance_sheet_df.drop(columns=['TTM'])
            balance_sheet_df.rename(columns = lambda x: x.split(' ')[1], inplace=True)
            balance_sheet_df['TTM']=TTM
        else:
            balance_sheet_df.rename(columns = lambda x: x.split(' ')[1], inplace=True)
        balance_sheet_df['Ticker']=symbol
        balance_sheet_df['Type']="Balance_Sheet"
        balance_sheet_df=balance_sheet_df.loc[:,~balance_sheet_df.columns.duplicated()]
        #balance_sheet_df['Company_Name']=company_name
        y_file_standalone=y_file_standalone.append(balance_sheet_df)
    except:
        print("Error Getting Balance Sheet Data for symbol:", symbol)
    try:
        cash_flow_table=content.find("section",{"id":'cash-flow'}).find_all('table')
        cash_flow_df = pd.read_html(str(cash_flow_table))[0]
        cash_flow_df.columns.values[0] = "KPI"
        cash_flow_df.set_index("KPI",inplace=True)
        column_list = cash_flow_df.columns.tolist()
        if 'TTM' in column_list :
            TTM=cash_flow_df['TTM']
            cash_flow_df=cash_flow_df.drop(columns=['TTM'])
            cash_flow_df.rename(columns = lambda x: x.split(' ')[1], inplace=True)
            cash_flow_df['TTM']=TTM
        else:
            cash_flow_df.rename(columns = lambda x: x.split(' ')[1], inplace=True)
        cash_flow_df['Ticker']=symbol
        cash_flow_df['Type']="Cash_Flow"
        cash_flow_df=cash_flow_df.loc[:,~cash_flow_df.columns.duplicated()]
        #cash_flow_df['Company_Name']=company_name
        y_file_standalone=y_file_standalone.append(cash_flow_df)   
    except:
        print("Error getting Cash Flow Data for symbol:", symbol)
    count=count+1
    driver.quit()
    time.sleep(1.5)

Now processing for Symbol:  1    INFY
Now processing for Symbol:  2    RELIANCE


In [24]:
z_file_standalone.to_csv(z_file_name_standalone)
q_file_standalone.to_csv(q_file_name_standalone)
y_file_standalone.to_csv(y_file_name_standalone)
s_file_standalone.to_csv(s_file_name_standalone)
z_file_standalone[z_file_standalone['Name']==z_file_standalone['Company_Name']].to_csv('data/Peer_Ticker_nifty500_stand_cleaned.csv')

# Save Cookie for Screener.in Login to Local Disk

In [25]:
def save_cookie(driver, path):
    with open(path, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)

def load_cookie(driver, path):
     with open(path, 'rb') as cookiesfile:
         cookies = pickle.load(cookiesfile)
         for cookie in cookies:
             driver.add_cookie(cookie)

In [26]:
# Pause here . A Page will open for entering your login which would then be saved
driver = selenium.webdriver.Chrome('C:\\Users\\sashhar\\Desktop\\Quantopian\\driver\\chromedriver.exe')
driver.get("https://www.screener.in/login/")

In [27]:
pickle.dump( driver.get_cookies() , open("cookie_use.pkl","wb"))
driver.quit()